In [1]:
from DDS_Seq import DDS, Seq
from DDS_config import *

import sys
import os
sys.path.append('..')

from compiler.Opfunc.OpfuncPulse import OpfuncPulse
from compiler.Opfunc.OpfuncRF_DDS import OpfuncRF_DDS

import re

def extract_number(string):
    # 使用正则表达式匹配所有数字
    numbers = re.findall(r'\d+', string)
    # 如果找到数字，返回第一个数字
    if numbers:
        return int(numbers[0])
    # 如果没有找到数字，返回None
    return None
# first, initialize the sequence
DDS.VERBOSE = True
# print(DDS.All)
# print(len(DDS.List))
# detection
# seq = Seq().S0(4)
seq = Seq().S0(4).S1(5)  # 序列
print(seq)
# rd_seq = seq.compile()
# print(seq.compile())
# print(seq.compile(4))
print("序列准备好了")
seqs = seq.seq
print(seqs[0][0])
print(seqs[0][0].name[0])
print(seqs[0][0][0])
print(seqs[0][0][1])
print(seqs[0][0][2])
print(extract_number("dds21"))
type(seqs[0][0])==type(DDS("dds"))

[[dds0.s0 = [100, 0.5, 0], dds1.s0 = [110, 0.6, 0], dds2.s0 = [120, 0.7, 0], dds3.s0 = [130, 0.8, 0], 2, 4], [dds0.s0 = [100, 0.5, 0], dds1.s0 = [110, 0.6, 0], dds2.s0 = [120, 0.7, 0], dds3.s0 = [130, 0.8, 0], dds12.s0 = [100, 0.2, 0], dds13.s0 = [100, 0.2, 0], dds14.s0 = [100, 0.2, 0], dds15.s0 = [100, 0.2, 0], 3, 5]]
序列准备好了
dds0.s0 = [100, 0.5, 0]
dds0
100
0.5
0
21


True

In [2]:
from typing import List
dds: List[OpfuncRF_DDS] = [OpfuncRF_DDS(DeviceID=i) for i in range(16)]
ttl: List[OpfuncPulse] = [OpfuncPulse(DeviceID=i) for i in range(16)]

In [3]:
def compile2Bell(seqs): # RTMQv1.0 transfer to Bell
    """
    将序列转换为Bell硬件执行的IR   
    每个dds通道有4个参数[f,a,p,t]
    每个ttl通道有3个参数[mode,address]，暂时用来计数input , [mode, delay]
    """
    seq_len = len(seqs)
    seq4Bell = []
    for seq in seqs:
        # 逆序遍历
        Flag_TTL = False
        Flag_Delay = False
        delay = 0
        DDSList = []
        for item in seq[::-1]:
            if type(item)==type(DDS("dds")):
                # print("dds")
                DeviceID = extract_number(item.name[0])
                Freq = item[0]
                Amp = item[1]
                Phase = item[2]
                DDSList.append(["dds",DeviceID, Freq, Amp, Phase,delay])
            elif Flag_Delay == False:
                delay = item # 读取延时
                Flag_Delay = True
                # print("delay:", delay)
            elif Flag_TTL==False: # 读取TTL
                TTL = item
                mode = 1
                # if mode == #TTL mode 在硬件中不能导入
                Flag_TTL = True

                # print("TTL channel:", TTL)
        DDSList.reverse() # 反转顺序
        DDSList.append(["TTL",TTL, mode, delay])
        # print("DDSList:", DDSList)
        seq4Bell.append(DDSList)
    # print("Bell序列:", seq4Bell)
    return seq4Bell
seqsIR = compile2Bell(seqs)
print("Bell序列:", seqsIR)

Bell序列: [[['dds', 0, 100, 0.5, 0, 4], ['dds', 1, 110, 0.6, 0, 4], ['dds', 2, 120, 0.7, 0, 4], ['dds', 3, 130, 0.8, 0, 4], ['TTL', 2, 1, 4]], [['dds', 0, 100, 0.5, 0, 5], ['dds', 1, 110, 0.6, 0, 5], ['dds', 2, 120, 0.7, 0, 5], ['dds', 3, 130, 0.8, 0, 5], ['dds', 12, 100, 0.2, 0, 5], ['dds', 13, 100, 0.2, 0, 5], ['dds', 14, 100, 0.2, 0, 5], ['dds', 15, 100, 0.2, 0, 5], ['TTL', 3, 1, 5]]]


In [4]:
# IR to Bell hardware 
"""
这个函数用来将IR数据结构转换成Bell机器码，暂不考虑跳转格式
"""
print(seqsIR)
print(seqsIR[0][4][3])

[[['dds', 0, 100, 0.5, 0, 4], ['dds', 1, 110, 0.6, 0, 4], ['dds', 2, 120, 0.7, 0, 4], ['dds', 3, 130, 0.8, 0, 4], ['TTL', 2, 1, 4]], [['dds', 0, 100, 0.5, 0, 5], ['dds', 1, 110, 0.6, 0, 5], ['dds', 2, 120, 0.7, 0, 5], ['dds', 3, 130, 0.8, 0, 5], ['dds', 12, 100, 0.2, 0, 5], ['dds', 13, 100, 0.2, 0, 5], ['dds', 14, 100, 0.2, 0, 5], ['dds', 15, 100, 0.2, 0, 5], ['TTL', 3, 1, 5]]]
4


In [ ]:
for i in range(16):
    dds[i].reset()
    # print(dds[i].read_state())

for seq in seqsIR:
    for item in seq:
        if item[0] == "dds":
            dds[item[1]].setwaveform(item[2], item[3], item[4], item[5])
        elif item[0] == "TTL":
            # ttl[item[1]].reset()
            ttl[item[1]].setwaveform("output",0,1,item[3]) #设置为输出模式

for i in range(4):
    # print(dds[i].read_state())
    print(dds[i].read_arrays())
    dds[i].gen_assembler()
    print(dds[i].read_arrays())
    dds[i].adjust_array_length()
    print(dds[i].read_arrays())

[]
[b'\x00\x00\x00\x05ffff\x00\x00\x00\x00\x00\x00\x80\x00']
[b'\x00\x00\x00\x05ffff\x00\x00\x00\x00\x00\x00\x80\x00', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
[]
[b'\x00\x00\x00\x05p\xa3\xd7\n\x00\x00\x00\x00\x00\x00\x99\x99']
[b'\x00\x00\x00\x05p\xa3\xd7\n\x00\x00\x00\x00\x00\x00\x99\x99', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
[]
[b'\x00\x00\x00\x05z\xe1G\xae\x00\x00\x00\x00\x00\x00\xb33']
[b'\x00\x00\x00\x05z\xe1G\xae\x00\x00\x00\x00\x00\x00\xb33', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
[]
[b'\x00\x00\x00\x05\x85\x1e\xb8Q\x00\x00\x00\x00\x00\x00\xcc\xcc']
[b'\x00\x00\x00\x05\x85\x1e\xb8Q\x00\x00\x00\x00\x00\x00\xcc\xcc', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']


In [8]:
# 针对一块板卡生成dds配置文件
data = []
for i in range(4):
    print(dds[i].read_arrays())
    array_128bit = dds[i].read_arrays()
    data.append(array_128bit)
print("data:", data)

[b'\x00\x00\x00\x05ffff\x00\x00\x00\x00\x00\x00\x80\x00', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
[b'\x00\x00\x00\x05p\xa3\xd7\n\x00\x00\x00\x00\x00\x00\x99\x99', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
[b'\x00\x00\x00\x05z\xe1G\xae\x00\x00\x00\x00\x00\x00\xb33', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
[b'\x00\x00\x00\x05\x85\x1e\xb8Q\x00\x00\x00\x00\x00\x00\xcc\xcc', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']
data: [[b'\x00\x00\x00\x05ffff\x00\x00\x00\x00\x00\x00\x80\x00', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'], [b'\x00\x00\x00\x05p\xa3\xd7\n\x00\x00\x00\x00\x00\x00\x99\x99', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'], [b'\x00\x00\x00\x05z\xe1G\xae\x00\x00\x00\x00\x00\x00\xb33', b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff'], [b'\x00\x00\x00\x05\x85\x1e\xb8Q\x00\x00\x00\x00\x00\x00\xcc\xcc', b'\xf

In [9]:
def merge_param_files_with_header(file1, output_file):
    with open(file1, 'r') as f1:
        lines1 = f1.readlines()
    total_lines = len(lines1)
    total_lines_hex = format(total_lines, '04x')
    header = f"eb9c55aa0002000000000000{total_lines_hex}0000\n"
    merged_content = [header] + lines1
    with open(output_file, 'w') as output:
        output.writelines(merged_content)
    print(f"文件合并完成！新文件已保存到: {output_file}")
    print(f"标头行: {header.strip()}")

In [ ]:
for i in range(4):
    dds[i].output2file("data/dds.txt") #在下面续写，而不是覆盖重写
merge_param_files_with_header("data/dds.txt", "data/ddswithheader.txt")

文件合并完成！新文件已保存到: data/ddswithheader.txt
标头行: eb9c55aa000200000000000000080000
